In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords
import re

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to C:\Users\Zain
[nltk_data]     Khalid\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
# Load your dataset
data = pd.read_csv('twitter.csv')

# Display the first few rows to understand the structure
print(data.head())


   id  label                                              tweet
0   1      0   @user when a father is dysfunctional and is s...
1   2      0  @user @user thanks for #lyft credit i can't us...
2   3      0                                bihday your majesty
3   4      0  #model   i love u take with u all the time in ...
4   5      0             factsguide: society now    #motivation


In [35]:
# Check for NaN values and drop them
data.dropna(subset=['label', 'tweet'], inplace=True)

# Ensure labels are integers
data['label'] = data['label'].astype(int)

# Display unique labels
print("Unique labels in dataset:", data['label'].unique())


Unique labels in dataset: [0 1]


In [36]:
# Tokenization
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Check the sequences
print("Number of sequences:", len(sequences))


Number of sequences: 31962


In [37]:
# Padding sequences
max_length = max(len(s) for s in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length)

# Check the shape of padded sequences
print("Shape of padded sequences:", padded_sequences.shape)


Shape of padded sequences: (31962, 39)


In [38]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

print("Training set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)


Training set shape: (25569, 39) (25569,)
Test set shape: (6393, 39) (6393,)


In [39]:
# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=100, input_length=max_length))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(len(set(labels)), activation='softmax'))  # Use softmax for multi-class classification


In [40]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [41]:
# Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/5
400/400 [==============================] - 103s 238ms/step - loss: 0.1749 - accuracy: 0.9448 - val_loss: 0.1206 - val_accuracy: 0.9567
Epoch 2/5
400/400 [==============================] - 75s 187ms/step - loss: 0.0919 - accuracy: 0.9670 - val_loss: 0.1191 - val_accuracy: 0.9589
Epoch 3/5
400/400 [==============================] - 65s 163ms/step - loss: 0.0702 - accuracy: 0.9749 - val_loss: 0.1352 - val_accuracy: 0.9587
Epoch 4/5
400/400 [==============================] - 73s 183ms/step - loss: 0.0519 - accuracy: 0.9824 - val_loss: 0.1451 - val_accuracy: 0.9581
Epoch 5/5
400/400 [==============================] - 65s 161ms/step - loss: 0.0403 - accuracy: 0.9863 - val_loss: 0.1713 - val_accuracy: 0.9520


In [42]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


200/200 [==============================] - 4s 20ms/step - loss: 0.1713 - accuracy: 0.9520
Test Loss: 0.17126740515232086
Test Accuracy: 0.9519787430763245


In [43]:
#Save the model
model.save('sentiment_analysis_model.h5')


C:\Users\Zain Khalid\anaconda3\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [44]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('sentiment_analysis_model.h5')


In [45]:
# Example tweets for prediction
new_tweets = [
    "This is the best product I've ever used!",
    "I am really disappointed with the service.",
    "It's okay, nothing special."
]

# Tokenization
new_sequences = tokenizer.texts_to_sequences(new_tweets)

# Padding sequences
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_length)


In [46]:
# Make predictions
predictions = model.predict(new_padded_sequences)

# Get the predicted class for each tweet
predicted_classes = np.argmax(predictions, axis=1)

# Print predictions
for tweet, predicted_class in zip(new_tweets, predicted_classes):
    print(f'Tweet: "{tweet}" => Predicted Class: {predicted_class}')


1/1 [==============================] - 1s 503ms/step
Tweet: "This is the best product I've ever used!" => Predicted Class: 0
Tweet: "I am really disappointed with the service." => Predicted Class: 0
Tweet: "It's okay, nothing special." => Predicted Class: 0
